In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import simpledialog
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
import numpy as np
import cv2
from cvzone.HandTrackingModule import HandDetector
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from googlesearch import search
import pyttsx3
from twilio.rest import Client
import geocoder
import matplotlib.pyplot as plt
import os 


class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Integrated Python Application")
        self.geometry("600x600")  # Increased height to accommodate the title and message
        self.configure(bg='black')  # Set background to black

        # Create title and welcome message
        self.title_label = tk.Label(self, text="Rudra Pratap Singh Shekhawat", bg='black', fg='white', font=('Algerian', 18, 'bold'))  # Algerian font
        self.title_label.pack(pady=(10, 5))

        self.welcome_label = tk.Label(self, text="Welcome to my project!", bg='black', fg='white', font=('Helvetica', 15))
        self.welcome_label.pack(pady=(0, 20))

        # Button styles
        button_style = {
            'bg': 'white',  # Set button background to white
            'fg': 'black',
            'font': ('Helvetica', 11),
            'width': 23,
            'height': 1
        }

        # Create buttons for different functionalities
        tk.Button(self, text="Hand Detector", command=self.fingerprint_detector, **button_style).pack(pady=10)
        tk.Button(self, text="Set Volume", command=self.set_volume, **button_style).pack(pady=10)
        tk.Button(self, text="Search Google", command=self.search_google, **button_style).pack(pady=10)
        tk.Button(self, text="Text to Speech", command=self.text_to_speech, **button_style).pack(pady=10)
        tk.Button(self, text="Capture Image", command=self.capture_image, **button_style).pack(pady=10)
        tk.Button(self, text="Apply Sunglasses Filter", command=self.apply_sunglasses_filter, **button_style).pack(pady=10)
        tk.Button(self, text="Send SMS", command=self.send_sms, **button_style).pack(pady=10)
        tk.Button(self, text="Get Location", command=self.get_location, **button_style).pack(pady=10)
        tk.Button(self, text="Apply Image Filters", command=self.apply_image_filters, **button_style).pack(pady=10)
        tk.Button(self, text="Create Custom Image", command=self.create_custom_image, **button_style).pack(pady=10)
        tk.Button(self, text="Crop Image", command=self.crop_image, **button_style).pack(pady=10)


    def set_volume(self):
        try:
            volume_level = simpledialog.askfloat("Volume Control", "Enter volume level (in dB):", minvalue=-96.0, maxvalue=0.0)
            if volume_level is not None:
                devices = AudioUtilities.GetSpeakers()
                interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
                volume = cast(interface, POINTER(IAudioEndpointVolume))
                volume.SetMasterVolumeLevel(volume_level, None)
                messagebox.showinfo("Volume Control", f"Volume set to {volume_level} dB")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to set volume: {e}")
    
    def search_google(self):
        query = simpledialog.askstring("Google Search", "Enter search query:")
        if query:
            results = search(query, num_results=5)
            results_str = "\n".join([f"{i + 1}. {result}" for i, result in enumerate(results)])
            messagebox.showinfo("Google Search Results", results_str)

    def text_to_speech(self):
        text = simpledialog.askstring("Text to Speech", "Enter text to speak:")
        if text:
            engine = pyttsx3.init()
            engine.say(text)
            engine.runAndWait()

    def capture_image(self):
        filename = "captured_image.jpg"
        cap = cv2.VideoCapture(0)
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                cv2.imwrite(filename, frame)
                messagebox.showinfo("Capture Image", f"Image saved as {filename}")
            else:
                messagebox.showerror("Error", "Could not capture an image.")
            cap.release()
        else:
            messagebox.showerror("Error", "Could not access the webcam.")
        cv2.destroyAllWindows()

    def apply_sunglasses_filter(self):
        image_path = filedialog.askopenfilename(filetypes=[("Image Files", ".jpg;.png")])
        if image_path:
            self._apply_realistic_sunglasses_filter(image_path)
            messagebox.showinfo("Sunglasses Filter", "Sunglasses filter applied and saved as 'realistic_sunglasses_filter.jpg'")

    def _apply_realistic_sunglasses_filter(self, image_path):
        image = Image.open(image_path).convert('RGBA')
        width, height = image.size
        overlay = Image.new('RGBA', image.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(overlay)
        glasses_width = width // 3
        glasses_height = height // 10
        bridge_width = width // 10
        bridge_height = glasses_height // 2
        left_eye_center = (width // 3, height // 3)
        right_eye_center = (2 * width // 3, height // 3)
        bridge_center = (width // 2, height // 3 + glasses_height // 2)
        left_bbox = [left_eye_center[0] - glasses_width // 2, left_eye_center[1] - glasses_height // 2,
                     left_eye_center[0] + glasses_width // 2, left_eye_center[1] + glasses_height // 2]
        draw.ellipse(left_bbox, fill=(0, 0, 0, 128))
        right_bbox = [right_eye_center[0] - glasses_width // 2, right_eye_center[1] - glasses_height // 2,
                      right_eye_center[0] + glasses_width // 2, right_eye_center[1] + glasses_height // 2]
        draw.ellipse(right_bbox, fill=(0, 0, 0, 128))
        bridge_bbox = [bridge_center[0] - bridge_width // 2, bridge_center[1] - bridge_height // 2,
                       bridge_center[0] + bridge_width // 2, bridge_center[1] + bridge_height // 2]
        draw.rectangle(bridge_bbox, fill=(0, 0, 0, 128))
        image_with_sunglasses = Image.alpha_composite(image, overlay)
        image_with_sunglasses = image_with_sunglasses.convert('RGB')
        image_with_sunglasses.save('realistic_sunglasses_filter.jpg')

    def send_sms(self):
        to_phone_number = simpledialog.askstring("Send SMS", "Enter recipient's phone number:")
        message_body = simpledialog.askstring("Send SMS", "Enter message body:")
        if to_phone_number and message_body:
            try:
                account_sid = 'your_SID'  # Replace with your Twilio Account SID
                auth_token = 'your_auth_token'  # Replace with your Twilio Auth Token
                from_phone_number = 'twilio_phone_number'  # Replace with your Twilio phone number
                client = Client(account_sid, auth_token)
                message = client.messages.create(
                    body=message_body,
                    from_=from_phone_number,
                    to=to_phone_number
                )
                messagebox.showinfo("Send SMS", f"Message sent with SID: {message.sid}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to send SMS: {e}")

    def fingerprint_detector(self):
        print("fingerprint detector")
        model = HandDetector()
        ca = cv2.VideoCapture(0)
        st , ph = ca.read()
        hand = model.findHands(ph)
        handphoto = hand[1]
        cv2.imshow("hii",ph)
        cv2.waitKey()
        cv2.destroyAllWindows()
    

    def get_location(self):
        g = geocoder.ip('me')
        location_info = f"Latitude: {g.latlng[0]}\nLongitude: {g.latlng[1]}\nAddress: {g.address}"
        messagebox.showinfo("Location", location_info)

    def apply_image_filters(self):
        image_path = filedialog.askopenfilename(filetypes=[("Image Files", ".jpg;.png")])
        if image_path:
            image = Image.open(image_path)
            blurred_image = image.filter(ImageFilter.BLUR)
            sharpened_image = image.filter(ImageFilter.SHARPEN)
            edge_enhanced_image = image.filter(ImageFilter.EDGE_ENHANCE)
            grayscale_image = image.convert("L")
            enhancer = ImageEnhance.Brightness(image)
            brightened_image = enhancer.enhance(1.5)
            plt.figure(figsize=(12, 8))
            plt.subplot(2, 3, 1)
            plt.imshow(image)
            plt.title("Original")
            plt.axis('off')
            plt.subplot(2, 3, 2)
            plt.imshow(blurred_image)
            plt.title("Blurred")
            plt.axis('off')
            plt.subplot(2, 3, 3)
            plt.imshow(sharpened_image)
            plt.title("Sharpened")
            plt.axis('off')
            plt.subplot(2, 3, 4)
            plt.imshow(edge_enhanced_image)
            plt.title("Edge Enhanced")
            plt.axis('off')
            plt.subplot(2, 3, 5)
            plt.imshow(grayscale_image, cmap='gray')
            plt.title("Grayscale")
            plt.axis('off')
            plt.subplot(2, 3, 6)
            plt.imshow(brightened_image)
            plt.title("Brightened")
            plt.axis('off')
            plt.show()

    def create_custom_image(self):
        color_image = np.zeros((100, 100, 3), dtype=np.uint8)
        color_image[40:60, 40:60, 0] = 255
        plt.imshow(color_image)
        plt.title("Custom Image")
        plt.axis('off')
        plt.show()

    def crop_image(self):
        image_path = filedialog.askopenfilename(filetypes=[("Image Files", ".jpg;.png")])
        if image_path:
            image = cv2.imread(image_path)
            height, width = image.shape[:2]
            crop_x = width // 4
            crop_y = height // 4
            crop_width = width // 2
            crop_height = height // 2
            cropped_image = image[crop_y:crop_y + crop_height, crop_x:crop_x + crop_width]
            cropped_image_path = "cropped_image.jpg"
            cv2.imwrite(cropped_image_path, cropped_image)
            messagebox.showinfo("Crop Image", f"Cropped image saved as {cropped_image_path}")
            cv2.imshow("Cropped Image", cropped_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

   
if __name__== "__main__":
    app = Application()
    app.mainloop()